In [1]:
# the intention of this script is to retrieve each university out of "College Rankings __ Attractiveness of the Students" + num + ".html" and store it in a list
# the list will be used to create an dataframe with the university and attractiveness
# the script will then find the state AND POPULATION of each university and store it with the university and attractiveness
# the script will then calculate each state's average attractiveness and store it in a dataframe
# this dataframe will be used to create a heatmap of the US with the average attractiveness of each state

In [2]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

In [134]:
universities = []
attractiveness = []

# retrieve the universities and attractiveness score from each html file

for i in range(1, 7):
    with open("College Rankings __ Attractiveness of the Students" + str(i) + ".html") as f:
        soup = BeautifulSoup(f, 'html.parser')
        universitiesBS = soup.find_all('a', class_='ratings_list')
        for university in universitiesBS:
            universities.append(university.string)
        attractivenessBS = soup.find_all('div', align='right')
        for attr in attractivenessBS:
            attractiveness.append(attr.string)


# clean the attractiveness list
# remove all the None values
attractiveness = [x for x in attractiveness if x is not None]

# remove all the elements which start with S, as they are not floats
attractiveness = [x for x in attractiveness if not x.startswith('S')]

#print(universities)
#print(attractiveness)

print(len(universities))
print(len(attractiveness))


505
505


In [135]:
# create a dataframe with the universities and their attractiveness
df = pd.DataFrame({"University": universities, "Attractiveness": attractiveness})

# replace "Malone College" with "Malone University" as it is now known
df.loc[df["University"] == "Malone College", "University"] = "Malone University"

# print the head and tail of the dataframe by combining them and printing them
pd.concat([df.head(), df.tail()])

,University,Attractiveness
0,Pepperdine University,89.7
1,Villanova University,87.8
2,Vanderbilt University,87.5
3,Texas Christian University,87.2
4,Southern Methodist University,87.0
500,Worcester Polytechnic Institute,28.2
501,"University of Missouri, Rolla",27.5
502,Polytechnic University,26.7
503,Rose-Hulman Institute of Technology,26.4
504,Clarkson University,24.1


In [104]:

# list of all the states in the US for simple regex matching (taken from https://gist.github.com/norcal82/e4c7e8113f377db184bb)
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]


In [105]:
def find_population_from_wiki(wiki_page_soup: BeautifulSoup):

    # get the table with the state and population
    table = None
    rows = None
    try:
        table = wiki_page_soup.find('table', class_='infobox vcard')
        rows = table.find_all('tr')
    except:
        print("No table found")
        return 0

    # iterate through the rows and find the population
    for row in rows:

        if len(row.find_all('th')) != 0:
            #print(row.find_all('th'))
            if row.find_all('th')[0].string == "Population" or row.find_all('th')[0].string == "Students":
                return int(re.search(r'\d{1,3}(,\d{3})*', row.find_all('td')[0].get_text()).group().replace(",", ""))
            if row.find_all('th')[0].string == "Undergraduates":
                #print(re.search(r'\d{1,3}(,\d{3})*', row.find_all('td')[0].get_text()).group())
                return int(int(re.search(r'\d{1,3}(,\d{3})*', row.find_all('td')[0].get_text()).group().replace(",", ""))*1.1)
    print("Population not found")
    return 0

    

In [128]:
df["University"] = universities

# create a list of the states of the universities
state = []
population = []

# create a list of the universities that couldn't be found
not_found = []

for i in range(len(df["University"])):

    if (len([x for x in state if x is not None]) != len(population)):
        print([x for x in state if x is not None])
        print([x for x in population if x != 0])
        print("Lengths of state and population lists do not match")
        break

    #print(df["University"][i])
    # check to see if one of the words in the university name is a state
    for word in df["University"][i].split():
        if word in state_names:
            state.append(word)
            break
        elif word == "SUNY" or word == "CUNY":
            state.append("New York")
            # replace the word "SUNY" or "CUNY" with the rest of the university name
            df.loc[i, "University"] = re.sub(r"SUNY ", "State University of New York ", df["University"][i])
            df.loc[i, "University"] = re.sub(r"CUNY ", "City University of New York ", df["University"][i])

            # remove the word "college " from the end of the university name
            df.loc[i, "University"] = re.sub(r" College", "", df["University"][i])
            break
        elif word == "York":
            state.append("New York")
            break
        elif word == "Rhode":
            state.append("Rhode Island")
            break
        else:
            state.append(None)

    url = "https://en.wikipedia.org/wiki/" + df["University"][i]
    # if the university has a space in its name, replace it with an underscore
    url = re.sub(r" ", "_", url)
    #print(url)
    # if the university has a parenthesis or a comma in its name, remove it along with the rest of the name
    url = re.sub(r"_\(.*", "", url)
    #url = re.sub(r",.*", "", url)
    # if the university has the words "Main Campus" in its name, remove it
    url = re.sub(r",_Main_Campus", "", url)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # if the page displays: "Wikipedia does not have an article with this exact name", then the university was not found
    if len(soup.find_all("table", id="noarticletext")) > 0:
        url = "https://en.wikipedia.org/wiki/" + df["University"][i]
        # if the university has a space in its name, replace it with an underscore
        url = re.sub(r" ", "_", url)
        #print(url)
        # if the university has a parenthesis or a comma in its name, remove it along with the rest of the name
        url = re.sub(r"_\(.*", "", url)
        url = re.sub(r",.*", "", url)
        # if the university has the words "Main Campus" in its name, remove it
        url = re.sub(r"Main_Campus", "", url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
    
    print(url)
        

    population.append(find_population_from_wiki(soup))
    print(population[-1])

    
    if state[-1] == None:
        # remove the None value from the very end of the list and continue to use wikipedia
        state = state[:-1]
    else:
        continue

    
    try:
        state.append(soup.find_all("div", class_="state")[0].string)
    except:
        try:
            state[-1] = soup.find_all("div", class_="locality")[0].string
            # if there's a comma+space in the locality, remove it along with everything before it
            state[-1] = re.sub(r".*, ", "", state[-1])
            print(str(int(i/len(df["University"])*1000)/10.0) + "%")
        except:
            state.append("N/A")
            population[-1] = "0"
            print("N/A: " + url)
            not_found.append(df["University"][i])

https://en.wikipedia.org/wiki/Pepperdine_University
10030
https://en.wikipedia.org/wiki/Villanova_University
9800
0.1%
https://en.wikipedia.org/wiki/Vanderbilt_University
13710
https://en.wikipedia.org/wiki/Texas_Christian_University
12785
https://en.wikipedia.org/wiki/Southern_Methodist_University
12053
https://en.wikipedia.org/wiki/Arizona_State_University
57588
https://en.wikipedia.org/wiki/Miami_University
19752
https://en.wikipedia.org/wiki/University_of_Mississippi_
24710
https://en.wikipedia.org/wiki/Coastal_Carolina_University
10484
https://en.wikipedia.org/wiki/University_of_California,_Santa_Barbara
26179
https://en.wikipedia.org/wiki/Rollins_College
3127
1.9%
https://en.wikipedia.org/wiki/University_of_Southern_California
49318
https://en.wikipedia.org/wiki/University_of_Arizona
53187
https://en.wikipedia.org/wiki/San_Diego_State_University
37539
https://en.wikipedia.org/wiki/Florida_State_University
45493
https://en.wikipedia.org/wiki/University_of_Colorado_at_Boulder
37153

In [130]:
# remove all None values from the state list

state = [x for x in state if x is not None]

# remove all the 0 values from the population list

#population = [x for x in population if x != 0]

print(len(population))

print(len(state))

#print(state)

print(len(not_found))
state_backup = state


505
505
19


In [131]:
for state_name in state:
    if state_name not in state_names and state_name != "N/A":
        print(state_name)
        if "D.C." in state_name:
            state[state.index(state_name)] = "District of Columbia"
        elif "New York" in state_name:
            state[state.index(state_name)] = "New York"

D.C.
New York (state)


In [132]:
# add the states to the dataframe
df["State"] = state

# add the population to the dataframe
df["Population"] = population

# save the dataframe to a csv file
df.to_csv("universitiesAttractivenessPopulation.csv")

# print them
df

,University,Attractiveness,State,Population
0,Pepperdine University,89.7,California,10030
1,Villanova University,87.8,Pennsylvania,9800
2,Vanderbilt University,87.5,Tennessee,13710
3,Texas Christian University,87.2,Texas,12785
4,Southern Methodist University,87.0,Texas,12053
...,...,...,...,...
500,Worcester Polytechnic Institute,28.2,Massachusetts,5770
501,"University of Missouri, Rolla",27.5,Missouri,6456
502,Polytechnic University,26.7,N/A,0
503,Rose-Hulman Institute of Technology,26.4,Indiana,2385


In [166]:
# load the dataframe from the csv file
df = pd.read_csv("universitiesAttractivenessPopulation.csv")

# calculate the average attractiveness based on each state and the population of the university
df["Attractiveness"] = df["Attractiveness"].astype(float)
df["Population"] = df["Population"].astype(int)
df["Attractiveness"] = df["Attractiveness"] * df["Population"]
df["Attractiveness"] = df["Attractiveness"].astype(float)
df["Number of Colleges"] = 1
df = df.groupby("State").sum()

df["Attractiveness"] = df["Attractiveness"] / df["Population"]

# create a new dataframe from only the state, attractiveness, and number of colleges columns
df = df[["Attractiveness", "Number of Colleges"]]

# sort the dataframe by the attractiveness
df = df.sort_values(by="Attractiveness", ascending=False)

df = df.reset_index()

# save the dataframe to a csv file
df.to_csv("averageAttractivenessByState.csv")

# print the dataframe
df



,State,Attractiveness,Number of Colleges
0,Mississippi,79.681092,4
1,Delaware,78.689081,2
2,Hawaii,77.600000,1
3,South Carolina,74.892107,6
4,Utah,73.649182,5
5,Arizona,73.547657,4
6,Vermont,72.900000,2
7,Florida,71.166491,20
8,Alabama,71.087463,8
9,North Carolina,70.538414,11


In [158]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [160]:
# add the two-letter state abbreviation to the dataframe
df["State Abbreviation"] = df["State"].map(us_state_to_abbrev)

In [162]:
# go through each state to ensure the state is a state
for i in range(len(df["State"])):
    if df["State"][i] not in us_state_to_abbrev.keys():
        print(df["State"][i])
        # if the state is not a state, remove it from the dataframe
        df = df.drop(i)

In [165]:


# use the plotly library to create a heatmap of the US with the average attractiveness of each state
import plotly.express as px
import nbformat

fig = px.choropleth(df, locations="State Abbreviation",
                    locationmode="USA-states",
                    color="Attractiveness",
                    scope="usa",
                    title="Average Attractiveness of College Students by State",
                    labels={"Attractiveness": "Average Attractiveness"},
                    color_continuous_scale=px.colors.sequential.Plasma)

fig.show()

# save the figure to an html file
fig.write_html("averageAttractivenessByState.html")
